In [1]:
import numpy as np
from scipy.optimize import fsolve

In [2]:
class EOS:

    def __init__(self, temperature, pressure):
        self.T = temperature
        self.P = pressure
    
    def __PengRobinson(self, ʋ, a, b, ω, Tc, R):
        T = self.T
        P = self.P

        def α(T, ω, Tc):
            Tr = T/Tc
            ωpol = 0.37464 + 1.5422*ω - 0.26992*np.power(ω, 2)
            alpha = np.power(1 + (1 - np.sqrt(Tr))*ωpol, 2)
            return alpha

        ʋpol = np.power(ʋ, 2) + 2*b*ʋ - np.power(b, 2)
        f = (R*T)/(ʋ - b) - a*α(T, ω, Tc)/ʋpol - P
        return T*ʋ + P*np.cos(ʋ)
    
    def solve_eos(self, ʋ0=1.0, a=1.5, b=1.8, ω=1.3, Tc=1.7, R=8.314):
        eos = lambda ʋ: self.__PengRobinson(ʋ, a, b, ω, Tc, R)
        ʋ_solution = fsolve(eos, ʋ0)
        return ʋ_solution
    

In [3]:
test = EOS(1, 5)
test.solve_eos()

array([1.97738303])

In [4]:
def my_func(ʋ, T, P):
    return T*ʋ + P*np.cos(ʋ)

root = fsolve(my_func, 1.5, args=(1, 5))
print(root)

[1.97738303]
